In [82]:
# Simple genetic algorithm example using Goldberg's monography
# "Genetic Algorithms in Search, Optimization and Machine Learning"
import numpy as np
import matplotlib.pyplot as plt
import math

In [40]:
# Fitness function for optimization
def fitness(_x):
    return _x**2

In [265]:
# Generate first population
def generate_population(X, population_power=4):
    new_population = []
    for chromosome in np.random.randint(low=1, high=X[-1], size=population_power):
        new_population.append([chromosome, fitness(chromosome)])
    return np.array(new_population)

In [281]:
def cross(chrome1, chrome2, k):
    new_chrome1 = chrome1[:k] + chrome2[k:]
    new_chrome2 = chrome2[:k] + chrome1[k:]
    return new_chrome1, new_chrome2

In [395]:
def invert(binary_in_str, k):
    inverted_gene = '1' if binary_in_str[k] == '0' else '0'
    return (binary_in_str[:k] + inverted_gene + binary_in_str[k+1:])

In [371]:
def reproduction(population, debug=True):
    # Calculate sum of function values
    _sum = sum(population[:,1])
    # Calculate number of copies of each chromosome in the next generation
    # based on its fitness and power of population
    copies_number = []
    probabilities = []
    for chromosome in population:
        probability = chromosome[1] / _sum
        probabilities.append(probability)
        copies_number.append(int(round(probability * len(population))))
    # Fill new generation with copies
    new_generation = []
    for i in range(len(population)):
        for j in range(copies_number[i]):
            new_generation.append(population[i])
    if debug is True:
        print("Copies: ", copies_number)
        print("Portions: ", ["%.1f%%" % (prob*100) for prob in probabilities])
    
    return np.array(new_generation)

In [399]:
def crossover(population, debug=True):
    # Form pairs for chromosomes:
    # number j on i-th index of 'pairs' list means
    # that i-th chromosome will crossover with j-th chromosome
    # in population given
    pairs = np.random.randint(1, len(population), len(population)//2)
    offsprings = []
    # Get each chromosome in binary (i.e. get gene form of each chromosome)
    parents = np.array([bin(chrome)[2:].zfill(6) for chrome in population[:,0]])
    genes_num = len(parents[0])
    if debug is True:
        print("Population: ", parents)
        print("Chromosomes' pairs: ")
        for i, p in zip(range(len(parents)), pairs):
            print("chrome[%d] x chrome[%d]" % (i, p))
    # Crossover
    for i in range(len(pairs)):
        # Generate crossover point k, k = [0, chromosome's len]
        crosspoint = int(np.random.randint(0, genes_num-1, 1))
        offspr_1, offspr_2 = cross(parents[i], parents[pairs[i]], k=crosspoint)
        offsprings.append(offspr_1)
        offsprings.append(offspr_2)
        if debug is True:
            print("%s crossover %s = %s(%d), %s(%d), k=%d" % (parents[i], parents[pairs[i]], 
                                                            offspr_1, int(offspr_1, 2), 
                                                            offspr_2, int(offspr_2, 2),
                                                            crosspoint))
    #return (np.array([int(offspr, 2) for offspr in offsprings]))
    return (offsprings)

In [410]:
def mutation(offsprings, mutation_probability=0.3, debug=True):
    # "Roll a dice" for every offspring to determine
    # if mutation will be applied to it
    probabilities = [float("%.2f"%(np.random.rand())) for offspring in offsprings]
    new_population = []
    mutation_counter = 0
    for offspring, probability in zip(offsprings, probabilities):
        # If generated probability <= real mutation prob,
        # generate gene index to invert and then invert it
        if probability <= mutation_probability:
            mutation_counter += 1
            k = int(np.random.randint(0, len(offspring), 1))
            offspring = invert(offspring, k)
        new_population.append(offspring)
    if debug is True:
        print(probabilities)
        print("Mutations in generation: ", mutation_counter)
    
    return new_population

In [268]:
# The task is to find x that will maximize fitness function Y = x^2, 
# i.e. ans = argmax(x, Y = x^2)
X = np.array(list(range(50)))

In [372]:
# Generate population of 5 chromosomes
size = 6
initial_population = generate_population(X, population_power=size)
initial_population
#population = [[13, '', 169],[24, '', 576],[8, '', 64],[19, '10011', 361]]

array([[  25,  625],
       [  37, 1369],
       [  24,  576],
       [  23,  529],
       [  20,  400],
       [   4,   16]])

In [373]:
#population = reproduction(initial_population, debug=True)
#print(population)

In [407]:
new_generation = crossover(initial_population)

Population:  ['011001' '100101' '011000' '010111' '010100' '000100']
Chromosomes' pairs: 
chrome[0] x chrome[4]
chrome[1] x chrome[1]
chrome[2] x chrome[2]
011001 crossover 010100 = 011100(28), 010001(17), k=3
100101 crossover 100101 = 100101(37), 100101(37), k=1
011000 crossover 011000 = 011000(24), 011000(24), k=1


In [411]:
print(new_generation)
print([int(chromosome, 2) for chromosome in new_generation])

['011100', '010001', '100101', '100101', '011000', '011000']
[28, 17, 37, 37, 24, 24]


In [412]:
mutated_generation = mutation(new_generation)
print(mutated_generation)
print([int(chromosome, 2) for chromosome in mutated_generation])

[0.69, 0.02, 0.26, 0.16, 0.42, 0.52]
Mutations in generation:  3
['011100', '110001', '100001', '100100', '011000', '011000']
[28, 49, 33, 36, 24, 24]


'0111'